# User’s Guide, Chapter 4: Lists, Streams (I) and Output
 
[Original Source](https://web.mit.edu/music21/doc/usersGuide/usersGuide_04_stream1.html#usersguide-04-stream1)  
Notebook Conversion: Charlotte Mertz, Antonia Mett  
Review: Sebastian Klassmann

## List of Contents
- [Working with multiple objects via Lists](#multipleobjects)
- [Introduction to Streams](#introductiontostreams)
- [Creating simple Streams](#simplestreams)
- [Accessing Streams](#accessingstreams)
- [Separating out elements by class with .getElementsByClass()](#separatingelements)
- [Separating out elements by offset with .getElementsByOffset()](#elementsbyoffset)
- [More Stream Features](#morefeatures)
- [Streams within Streams](#streamswithinstreams)
  
### Working with multiple objects via Lists<a name="multipleobjects"></a>  
Python has ways of working with multiple objects and music21 extends those ways to be more musical. Let’s look at how Python does it first and then see how music21 extends these ways.  

In [ ]:
from music21 import *
environment.keys()
environment.set('musicxmlPath', value='/usr/bin/musescore-xvfb')
environment.set('musescoreDirectPNGPath', value='/usr/bin/musescore-xvfb')

Say you have two notes, a C and an F# in the middle of the treble staff. Let's create those notes:

In [ ]:
note1 = note.Note("C4")
note2 = note.Note("F#4")

Let’s make the first note a half note by modifying its duration (by default all Note objects are quarter notes):

In [ ]:
note1.duration.type = 'half'
note1.duration.quarterLength

In [ ]:
note2.duration.quarterLength

To print the step (that is, the name without any octave or accidental information) of each of these notes, you could do something like this:

In [ ]:
print(note1.step)

In [ ]:
print(note2.step)

But suppose you had thirty notes? Then it’d be a pain to type “print(noteX.step)” thirty times. Fortunately, there’s a solution: we can put each of the note objects into a List which is a built in Python object that stores multiple other objects (like Notes or Chords, or even things like numbers). To create a list in Python, put square brackets ([]) around the things that you want to put in the list, separated by commas. Let’s create a list called noteList that contains note1 and note2:

In [ ]:
noteList = [note1, note2]

We can check that noteList contains our Notes by printing it:

In [ ]:
print(noteList)

The list is represented by the square brackets around the end with the comma in between them, just like how they were created originally.

Now we can write a two-line program that will print the step of each note in noteList. Most modern languages have a way of doing some action for each member (“element”) in a list (also called an “array” or sometimes “row”). In Python this is the “for” command. When you type these lines, make sure to type the spaces at the start of the second line. (When you’re done typing print(thisNote.step), you’ll probably have to hit enter twice to see the results.)

In [ ]:
for thisNote in noteList:
    print(thisNote.step)

 **for** thisNote **in** noteList:  

says that Python should take each note in noteList in order and temporarily call that note “thisNote” (but not note because note is the name of a module). Then the “:” at the end of the line indicates that everything that happens for a bit will apply to every Note in noteList one at a time. How does Python know when “a bit” is over? Simple: every line that is a part of the loop needs to be indented by putting in some spaces.  

Loops don’t save much time here, but imagine if noteList had dozens or hundreds of Notes in it? Then the ability to do something to each object becomes more and more important.  

Let’s add another note to noteList. First let’s create another note, a low B-flat:

In [ ]:
note3 = note.Note("B-2")

Then we’ll append that note to the end of noteList:

In [ ]:
noteList.append(note3)

Lists can be manipulated or changed. They are called “mutable” objects (we’ll learn about immutable objects later). Streams, as we will see, can be manipulated the same way through .append().

We can see that the length of noteList is now 3 using the len() function:

In [ ]:
len(noteList)

And if we write our looping function again, we will get a third note:

In [ ]:
for thisNote in noteList:
    print(thisNote.step)

We can find out what the first note of noteList is by writing:

In [ ]:
noteList[0]

Notice that in a list, the first element is [0], not [1]. (Think of it like how floors are numbered in European buildings compared to American buildings.)  

If we go forward one note, to the second note, we write:

In [ ]:
noteList[1]

We can also ask noteList where is note2 within it, using the index() method:

In [ ]:
noteList.index(note2)

If we want to get the last element of a list, we can write:

In [ ]:
noteList[-1]

Which is how basements are numbered in Europe as well. This is the same element as noteList[2] (our third Note), as we can have Python prove:

In [ ]:
noteList[-1] is noteList[2]

Lists will become important tools in your programming, but they don’t know anything about music. To get some intelligence into our music we’ll need to know about a music21 object similar to lists, called a Stream.

### Introduction to Streams<a name="introductiontostreams"></a>

The Stream object and its subclasses (Score, Part, Measure) are the fundamental containers for music21 objects such as Note, Chord, Clef, TimeSignature objects.

A container is like a Python list (or an array in some languages).

Objects stored in a Stream are generally spaced in time; each stored object has an offset usually representing how many quarter notes it lies from the beginning of the Stream. For instance in a 4/4 measure of two half notes, the first note will be at offset 0.0, and the second at offset 2.0.

Streams, further, can store other Streams, permitting a wide variety of nested, ordered, and timed structures. These stored streams also have offsets. So if we put two 4/4 Measure objects (subclasses of Stream) into a Part (also a type of Stream), then the first measure will be at offset 0.0 and the second measure will be at offset 4.0.

A critical feature of music21’s design that distinguishes it from other music analysis frameworks is that one music21 object can be simultaneously stored (or, more accurately, referenced) in more than one Stream. For examples, we might have numerous Measure Streams contained in a Part Stream. If we extract a region of this Part (using the measures() method), we get a new Stream containing the specified Measures and the contained notes. We have not actually created new notes within these extracted measures; the output Stream simply has references to the same objects. Changes made to Notes in this output Stream will be simultaneously reflected in Notes in the source Part. There is one limitation though: the same object should not appear twice in one hierarchical structure of Streams. For instance, you should not put a note object in both measure 3 and measure 5 of the same piece – it can appear in measure 3 of one piece and measure 5 of another piece.

### Creating simple Streams<a name="simplestreams"></a>
Objects stored in Streams are called elements and must be some type of Music21Object (don’t worry, almost everything in music21 is a Music21Object, such as Note, Chord, TimeSignature, etc.).

(If you want to put an object that’s not a Music21Object in a Stream, put it in an ElementWrapper.)

Streams are similar to Python lists in that they hold individual elements in order. They’re different in that they can only hold music21 objects such as Notes or Clef objects. But they’re a lot smarter and more powerful.

To create a Stream you’ll need to type stream.Stream() and assign it to a variable using the equal sign. Let’s call our Stream stream1:

In [ ]:
stream1 = stream.Stream()

Notice that just like how the (capital) Note object lives in a module called (lowercase) note, the (capital) Stream object lives in a module called (lowercase) stream. Variable names, like stream1 can be either uppercase or lowercase, but I tend to use lowercase variable names (or camelCase like we did with noteList).
The most common use of Streams is as places to store Notes. So let’s do just that: we can add the three Note objects we created above by using the append method of Stream:

In [ ]:
stream1.append(note1)
stream1.append(note2)
stream1.append(note3)

Of course, this would be a pain to type for hundreds of Notes, so we could also use the Stream method repeatAppend() to add a number of independent, unique copies of the same Note. This creates independent copies (using Python’s copy.deepcopy function) of the supplied object, not references.

In [ ]:
stream2 = stream.Stream()
n3 = note.Note('D#5') # octave values can be included in creation arguments
stream2.repeatAppend(n3, 4)
stream2.show()

Going back to our first stream, we can see that it has three notes, using len() :

In [ ]:
len(stream1)

Alternatively, we can use the show() method called as show('text') to see what is in the Stream and what its offset is (here 0.0, since we put it at the end of an empty stream).

In [ ]:
stream1.show('text')

In [ ]:
stream1.show()

### Accessing Streams<a name="accessingstreams"></a>  

We can also dive deeper into streams. Let’s get the step of each Note using the **for** thisNote **in** ...: command.  

But now we’ll use stream1 instead of noteList:

In [ ]:
for thisNote in stream1:
    print(thisNote.step)

And we can get the first and the last Note in a Stream by using the [X] form, just like other Python list-like objects:

In [ ]:
stream1[0]

In [ ]:
stream1[-1].nameWithOctave

While full list-like functionality of the Stream isn’t there, some additional methods familiar to users of Python lists are also available. The Stream index() method can be used to get the first-encountered index of a supplied object.

In [ ]:
note3Index = stream1.index(note3)
note3Index

Given an index, an element from the Stream can be removed with the pop() method.

In [ ]:
stream1.pop(note3Index)
stream1.show()

Since we removed note3 from stream1 with the the pop() method, let’s add note3 back into stream1 so that we can continue with the examples below using stream1 as we originally created it.

In [ ]:
stream1.append(note3)
stream1.show()

### Separating out elements by class with .getElementsByClass()<a name="separatingelements"></a>  
We can also gather elements based on the class (object type) of the element, by offset range, or by specific identifiers attached to the element. Gathering elements from a Stream based on the class of the element provides a way to filter the Stream for desired types of objects. The getElementsByClass() method iterates over a Stream of elements that are instances or subclasses of the provided classes. The example below gathers all Note objects and then all Rest objects. The easiest way to do this is to use for loops with .getElementsByClass():

In [ ]:
for thisNote in stream1.getElementsByClass(note.Note):
    print(thisNote, thisNote.offset)

If you want instead of passing the class note.Note you could instead pass the string "Note".

In [ ]:
for thisNote in stream1.getElementsByClass("Note"):
    print(thisNote, thisNote.offset)

It is also possible to pass in a list of classes or strings of class names to .getElementsByClass() which will return anything that matches any of the classes. Notice the [] marks in the next call, indicating that we are creating a list to pass to .getElementsByClass():

In [ ]:
for thisNote in stream1.getElementsByClass(["Note", "Rest"]):
    print(thisNote, thisNote.offset)

Since there are no note.Rest objects, it’s the same as above.

music21 has a couple of shortcuts that are equivalent to .getElementsByClass. For instance .notes is equivalent to .getElementsByClass(["Note", "Chord"]):

In [ ]:
for thisNote in stream1.notes:
    print(thisNote)

And .notesAndRests is equivalent to .getElementsByClass(['Note', 'Chord', 'Rest']).

In [ ]:
for thisNote in stream1.notesAndRests:
    print(thisNote)

Finally, there’s something slightly different. .pitches begins with a call to .notes, but then returns a list of all the pitches from every Note or Chord in the Stream:

In [ ]:
listOut = stream1.pitches
listOut

The result of a .getElementsByClass are not technically streams, but you can still call .show() on it:

In [ ]:
sOut = stream1.getElementsByClass(note.Note)
sOut.show('text')

But if you want to be absolutely sure, put the expression .stream() after it:

In [ ]:
sOut = stream1.getElementsByClass(note.Note).stream() # <-- different
sOut.show('text')

### Separating out elements by offset with .getElementsByOffset()<a name="elementsbyoffset"></a>  

The getElementsByOffset() method returns a Stream of all elements that fall either at a single offset or within a range of two offsets provided as an argument. In both cases a Stream is returned.

In [ ]:
sOut = stream1.getElementsByOffset(3)
len(sOut)

In [ ]:
sOut[0]

Like with .getElementsByClass() if you want a Stream from .getElementsByOffset(), add .stream() to the end of it.

In [ ]:
sOut = stream1.getElementsByOffset(2, 3).stream()
sOut.show('text')

We will do more with .getElementsByOffset() later when we also talk about getElementAtOrBefore() and getElementAfterElement()

### More Stream Features<a name="morefeatures"></a>  

Okay, so far we’ve seen that Streams can do the same things as lists, but can they do more? Let’s call the analyze method on stream to get the ambitus (that is, the range from the lowest note to the highest note) of the Notes in the Stream:

In [ ]:
stream1.analyze('ambitus')

Let’s take a second to check this. Our lowest note is note3 (B-flat in octave 2) and our highest note is note2 (F-sharp in octave 4). From B-flat to the F-sharp above it, is an augmented fifth. An augmented fifth plus an octave is an augmented twelfth. So we’re doing well so far. 

As we mentioned earlier, when placed in a Stream, Notes and other elements also have an offset (stored in .offset) that describes their position from the beginning of the stream. These offset values are also given in quarter-lengths (QLs).

Once a Note is in a Stream, we can ask for the offset of the Notes (or anything else) in it. The offset is the position of a Note relative to the start of the Stream measured in quarter notes. So note1’s offset will be 0.0, since it’s at the start of the Stream:

In [ ]:
note1.offset

note2’s offset will be 2.0, since note1 is a half note, worth two quarter notes:

In [ ]:
note2.offset

And note3, which follows the quarter note note2 will be at offset 3.0:

In [ ]:
note3.offset

(If we made note2 an eighth note, then note3’s offset would be the floating point [decimal] value 2.5. But we didn’t.) So now when we’re looping we can see the offset of each note. Let’s print the note’s offset followed by its name by putting .offset and .name in the same line, separated by a comma:

In [ ]:
for thisNote in stream1:
    print(thisNote.offset, thisNote.name)

(Digression: It’s probably not too early to learn that a safer form of .offset is .getOffsetBySite(stream1):

In [ ]:
note2.offset

In [ ]:
note2.getOffsetBySite(stream1)

What’s the difference? Remember how I said that .offset refers to the number of quarter notes that the Note is from the front of a Stream? Well, eventually you may put the same Note in different places in multiple Streams, so the .getOffsetBySite(X) command is a safer way that specifies exactly which Stream we are talking about. End of digression…)

As a final note about offsets, the lowestOffset property returns the minimum of all offsets for all elements on the Stream.

In [ ]:
stream1.lowestOffset

So, what else can we do with Streams? Like Note objects, we can show() them in a couple of different ways. Let’s hear these three Notes as a MIDI file:

In [ ]:
stream1.show('midi')

Or let’s see them as a score:

In [ ]:
stream1.show()

You might ask why is the piece in common-time (4/4)? This is just the default for new pieces, which is in the defaults module:

In [ ]:
defaults.meterNumerator

In [ ]:
defaults.meterDenominator

(Some of these examples use a system that automatically tries to get an appropriate time signature and appropriate clef; in this case, music21 figured out that that low B-flat would be easier to see in bass clef than treble.)

If you don’t have MIDI or MusicXML configured yet (we’ll get to it in a second) and you don’t want to have other programs open up, you can show a Stream in text in your editor:

In [ ]:
stream1.show('text')

This display shows the offset for each element (that is, each object in the Stream) along with what class it is, and a little bit more helpful information. The information is the same as what’s called the __repr__ (representation) of the object, which is what you get if you type its variable name at the prompt:

In [ ]:
note1

By the way, Streams have a __repr__ as well:

In [ ]:
stream1

that number at the end is the hex form of the .id of the Stream, which is a way of identifying it. Often the .id of a Stream will be the name of the Part (“Violin II”), but if it’s undefined then a somewhat random number is used (actually the location of the Stream in your computer’s memory). We can change the .id of a Stream:

In [ ]:
stream1.id = 'some_notes'
stream1

We could have also changed the .id of any of our Note objects, but it doesn’t show up in the Note’s __repr__:

In [ ]:
note1.id = 'my_favorite_C'
note1

Now, a Stream is a Music21Object just like a Note is. This is why it has an .id attribute and, more importantly, why you can call .show() on it.

What else makes a Music21Object what it is? It has a .duration attribute which stores a Duration object:

In [ ]:
stream1.duration

In [ ]:
stream1.duration.type

In [ ]:
stream1.duration.quarterLength

(Notice that the len() of a Stream, which stands for “length”, is not the same as the duration. the len() of a Stream is the number of objects stored in it, so len(stream1) is 3).

A related concept to the .duration of a Stream is its .highestTime, which is the time at which the latest element in the Stream ends. Usually this is the last element of the stream’s .offset plus its .quarterLength.

In [ ]:
stream1.highestTime

### Streams within Streams<a name="streamswithinstreams"></a>

And, as a Music21Object, a Stream can be placed inside of another Stream object. Let’s create a stream, called biggerStream (for reasons that will become obvious), that holds a Note D# at the beginning

In [ ]:
biggerStream = stream.Stream()
note2 = note.Note("D#5")
biggerStream.insert(0, note2)

Now we use the .append functionality to put stream1 at the end of biggerStream:

In [ ]:
biggerStream.append(stream1)

Notice that when we call .show('text') on biggerStream, we see not only the presence of note2 and stream1 but also all the contents of stream1 as well:

In [ ]:
biggerStream.show('text')

In [ ]:
biggerStream.show()

Notice though that the offsets, the little numbers inside curly brackets, for the elements of stream1 (“some notes”) relate only to their positions within stream1, not to their position within biggerStream. This is because each Music21Object knows its offset only in relation to its containing Stream, not necessarily to the Stream containing that Stream.

Also notice that note1 knows that it is in stream1 but doesn’t know that it is somewhere inside biggerStream:

In [ ]:
note1 in stream1

In [ ]:
note1 in biggerStream

All this might not seem like much of a big deal, until we tell you that in music21, Scores are made up of Streams within Streams within Streams. So if you have an orchestral score, it is a Stream, and the viola part is a Stream in that Stream, and measure 5 of the viola part is a Stream within that Stream, and, if there were a ‘’divisi’‘, then each ‘’diviso’’ voice would be a Stream within that Stream. Each of these Streams has a special name and its own class (Score, Part, Measure, and Voice), but they are all types of Streams.

So how do we find note1 inside biggerStream? That’s what the next two chapters are about.